In [8]:
import numpy as np                     # for random number generation
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
"""
Generated a set of fake data of value of a fund index with 365 daily quote each year, 
from date 2015.12.31 to 2036.12.31. The value of fund index as of 2015.12.31 is 100.00.
The values van be generated randomly but:
(Fund A,1) During 2016.01.01 to 2025.12.31 the average annual return is 15%
    and the volatility, standard deviation of annual return is 18%
(Fund A,2) During 2026.01.01 to 2036.12.31 the average annual return is 10%
    and the volatility, standard deviation of annual return is 11%
(Fund B,1) During 2016.01.01 to 2025.12.31 the average annual return is 10%
    and the volatility, standard deviation of annual return is 12%
(Fund B,2) During 2026.01.01 to 2036.12.31 the average annual return is 8%
    and the volatility, standard deviation of annual return is 10%
(Fund C,1) During 2016.01.01 to 2025.12.31 the average annual return is 18%
    and the volatility, standard deviation of annual return is 22%
(Fund C,2) During 2026.01.01 to 2036.12.31 the average annual return is 15%
    and the volatility, standard deviation of annual return is 18%

"""
# Generate date range from 2015-12-31 to 2036-12-31 (includes leap days automatically)

import numpy as np
import pandas as pd
from datetime import datetime

# ───────────────────────────────────────────────────────────────
# CONFIGURATION - tuned to approximate your target CAGRs
# ───────────────────────────────────────────────────────────────

START_DATE = datetime(2015, 12, 31)
END_DATE   = datetime(2036, 12, 31)
BASE_VALUE = 100.00

CHANGE_DATE = datetime(2026, 1, 1)

fund_configs = {
    'A': {
        '2016-2025': {'return': 0.142, 'vol': 0.175},
        '2026-2036': {'return': 0.153, 'vol': 0.105}
    },
    'B': {
        '2016-2025': {'return': 0.122, 'vol': 0.115},
        '2026-2036': {'return': 0.090, 'vol': 0.095}
    },
    'C': {
        '2016-2025': {'return': 0.245, 'vol': 0.215},
        '2026-2036': {'return': 0.044, 'vol': 0.175}
    },
    'D': {
        'whole_period': {'return': -0.057, 'vol': 0.007}
    }
}

# ───────────────────────────────────────────────────────────────
# Generate daily prices
# ───────────────────────────────────────────────────────────────

dates = pd.date_range(START_DATE, END_DATE, freq='D')
prices = {}

np.random.seed(42)

for fund_name, config in fund_configs.items():
    price_series = pd.Series(index=dates, dtype=float)
    price_series.iloc[0] = BASE_VALUE

    if fund_name == 'D':
        r = config['whole_period']['return']
        v = config['whole_period']['vol']
        mu_annual = np.log(1 + r) - 0.5 * v**2
        mu_daily = mu_annual / 365
        sigma_daily = v / np.sqrt(365)

        for i in range(1, len(dates)):
            daily_log_ret = np.random.normal(mu_daily, sigma_daily)
            daily_log_ret = max(daily_log_ret, 0)
            price_series.iloc[i] = price_series.iloc[i-1] * np.exp(daily_log_ret)
    else:
        regimes = {}
        for period, params in config.items():
            r = params['return']
            v = params['vol']
            mu_annual = np.log(1 + r) - 0.5 * v**2
            regimes[period] = {
                'mu_daily': mu_annual / 365,
                'sigma_daily': v / np.sqrt(365)
            }

        for i in range(1, len(dates)):
            dt = dates[i]
            if dt < CHANGE_DATE:
                mu_d = regimes['2016-2025']['mu_daily']
                sigma_d = regimes['2016-2025']['sigma_daily']
            else:
                mu_d = regimes['2026-2036']['mu_daily']
                sigma_d = regimes['2026-2036']['sigma_daily']
            
            log_ret = np.random.normal(mu_d, sigma_d)
            price_series.iloc[i] = price_series.iloc[i-1] * np.exp(log_ret)

    prices[fund_name] = price_series

# ───────────────────────────────────────────────────────────────
# Export full daily data to CSV
# ───────────────────────────────────────────────────────────────

df_daily = pd.DataFrame({
    'Date': dates.strftime('%Y-%m-%d'),
    'Fund_A': prices['A'].round(2),
    'Fund_B': prices['B'].round(2),
    'Fund_C': prices['C'].round(2),
    'Fund_D': prices['D'].round(2)
})

df_daily.to_csv("funds_daily_2015_to_2036.csv", index=False)
print("Full daily data exported to: funds_daily_2015_to_2036.csv\n")

# ───────────────────────────────────────────────────────────────
# Print end values at year-end 2015, 2025, 2036
# ───────────────────────────────────────────────────────────────

print("Fund Values at Year-End:")
print("══════════════════════════════════════════════════════════════")
print(f"{'Date':<12} {'Fund A':>12} {'Fund B':>12} {'Fund C':>12} {'Fund D':>12}")
print("══════════════════════════════════════════════════════════════")

for year in [2015, 2025, 2036]:
    dt = datetime(year, 12, 31)
    date_str = dt.strftime('%Y-%m-%d')
    try:
        va = prices['A'].loc[dt]
        vb = prices['B'].loc[dt]
        vc = prices['C'].loc[dt]
        vd = prices['D'].loc[dt]
        print(f"{date_str:<12} {va:12.2f} {vb:12.2f} {vc:12.2f} {vd:12.2f}")
    except KeyError:
        print(f"{date_str:<12} {'Data not available':>48}")

print("══════════════════════════════════════════════════════════════\n")

# ───────────────────────────────────────────────────────────────
# Print CAGRs for the three requested periods
# ───────────────────────────────────────────────────────────────

def cagr(start, end, years):
    if start <= 0 or end <= 0:
        return 0.00
    return ((end / start) ** (1 / years) - 1) * 100

dt_2025 = datetime(2025, 12, 31)

print("=== CAGRs for 2016-2036 (21 years) ===")
for fund in ['A', 'B', 'C', 'D']:
    start = prices[fund].iloc[0]
    end = prices[fund].iloc[-1]
    rate = cagr(start, end, 21)
    print(f"Fund {fund}: {rate:5.2f}% (End Value: {end:,.2f})")

print("\n=== CAGRs for 2016-2025 (10 years) ===")
for fund in ['A', 'B', 'C', 'D']:
    start = prices[fund].iloc[0]
    end = prices[fund].loc[dt_2025]
    rate = cagr(start, end, 10)
    print(f"Fund {fund}: {rate:5.2f}% (End Value: {end:,.2f})")

print("\n=== CAGRs for 2026-2036 (11 years) ===")
for fund in ['A', 'B', 'C', 'D']:
    start = prices[fund].loc[dt_2025]
    end = prices[fund].iloc[-1]
    rate = cagr(start, end, 11)
    print(f"Fund {fund}: {rate:5.2f}% (End Value: {end:,.2f})")

Full daily data exported to: funds_daily_2015_to_2036.csv

Fund Values at Year-End:
══════════════════════════════════════════════════════════════
Date               Fund A       Fund B       Fund C       Fund D
══════════════════════════════════════════════════════════════
2015-12-31         100.00       100.00       100.00       100.00
2025-12-31         761.45       282.69      1459.14       132.15
2036-12-31        1508.99      1044.09      1588.47       181.23
══════════════════════════════════════════════════════════════

=== CAGRs for 2016-2036 (21 years) ===
Fund A: 13.80% (End Value: 1,508.99)
Fund B: 11.82% (End Value: 1,044.09)
Fund C: 14.07% (End Value: 1,588.47)
Fund D:  2.87% (End Value: 181.23)

=== CAGRs for 2016-2025 (10 years) ===
Fund A: 22.51% (End Value: 761.45)
Fund B: 10.95% (End Value: 282.69)
Fund C: 30.74% (End Value: 1,459.14)
Fund D:  2.83% (End Value: 132.15)

=== CAGRs for 2026-2036 (11 years) ===
Fund A:  6.42% (End Value: 1,508.99)
Fund B: 12.61% (End Va